In [9]:
import os
import requests
import datetime
import pandas as pd


def GRIB_to_CSV(points, start_date, end_date):
    
    gribs_dir = 'GRIBS/'
    gribs_raw_dir = gribs_dir + 'Raw/'
    gribs_selected_dir = gribs_dir + 'Selected/'
    tabs_selected_dir = 'TABS/'
    csv_dir = 'METCOMP_CSV/'
    
    if os.path.isdir(gribs_dir):
        print(gribs_dir + ' exists.')
        
        # Deal with if when subfolders don't exist but GRIBS/ does.
        if os.path.isdir(gribs_raw_dir):
            pass
        else:
            print('Creating ' + gribs_raw_dir + '.')
            os.mkdir(gribs_raw_dir)
            
        if os.path.isdir(gribs_selected_dir):
            pass
        else:
            print('Creating ' + gribs_selected_dir + '.')
            os.mkdir(gribs_selected_dir)
    else:
        print('Creating ' + gribs_dir + '.')
        os.mkdir('GRIBS/')
        print('Creating ' + gribs_raw_dir + '.')
        os.mkdir('GRIBS/Raw')
        print('Creating ' + gribs_selected_dir + '.')
        os.mkdir('GRIBS/Selected')
        
    if os.path.isdir(tabs_selected_dir):
        print(tabs_selected_dir + ' exists.')
    else:
        print('Creating ' + tabs_selected_dir)
        os.mkdir(tabs_selected_dir)
    
    
    if os.path.isdir(csv_dir):
        print(csv_dir + ' exists.')
    else:
        print('Creating ' + csv_dir)
        os.mkdir(csv_dir)
    
    # Write filter file.
    with open('filter.txt', 'w') as filter_file:
        filter_file.write('write "GRIBS/Selected/selected_[shortName]_[levelType].grib";')
    
    
    # Loop over each day (Include end_date).
    current_date = start_date
    for n in range(0, (end_date - start_date + datetime.timedelta(days=1)).days):
    
        date_str = current_date.strftime('%Y-%m-%d')
        
        #print(date_str)
        
        # Build timestamps flag dict:
        timestamps = {}
        for h in range(0, 24):
            hour_str = ''
            if h < 10:
                hour_str = '0' + str(h)
            else:
                hour_str = str(h)
            
            tmp_ts_str = date_str + 'T' + hour_str + ':00:00Z'
            timestamps[tmp_ts_str] = False
        
        
        # Find all GRIB-files
        raw_files = []
        for r, d, f in os.walk(gribs_raw_dir):
            for file in f:
                if '.grib' in file:
                    raw_files.append(file)
        print(raw_files)
        current_file = 'MESAN_' + date_str + '.grib'
        if current_file not in raw_files:
            # Loop over each hour.
            print('Building GRIB-file.')
            with open(gribs_raw_dir + current_file, 'ab') as grib_file:
                for ts in timestamps:

                    #print(ts)
                    hour_str = ts.split('T')[1].split(':')[0]
                    date_str = ts.split('T')[0]
                    #print(date_str)
                    #print(hour_str)

                    # Fetch data.
                    url = 'https://opendata-download-grid-archive.smhi.se/data/6/'+ current_date.strftime("%Y%m") + '/MESAN_' + current_date.strftime("%Y%m%d") + hour_str + '00+000H00M'
                    print(url)

                    try:
                        # If successful fetch, flag fetch as 'good'.
                        grib_data = requests.get(url, allow_redirects=True)
                        timestamps[ts] = True
                    except requests.exceptions.RequestException:
                        # Handle unsuccessful fetch somehow.
                        print('REQUESTS EXCEPTION!!!')
                        continue

                    # Append to GRIB-file.
                    grib_file.write(grib_data.content)
        else:
            print('Found previously written file.')
            for ts in timestamps:
                timestamps[ts] = True
        
        # At this point, we have a GRIB file for current_date with 1h resolution.
        
        # Clear GRIBS/Selected/ perhaps to guarantee that no old
        # selected parameters are appertaining to previous dates.
        # Basically, all selected files should correspond to current_date.
        
        #print('Running grib_filter.')
        # Run bash
        grib_file_name = gribs_raw_dir + 'MESAN_' + date_str + '.grib'
        !grib_filter filter.txt {grib_file_name}
        
        # Find all files in GRIBS/Selected/.
        grib_files = []
        for r, d, f in os.walk(gribs_selected_dir):
            for file in f:
                if '.grib' in file:
                    grib_files.append(file)
        
        # For all selected_PARAM.grib files, create corresponding tabular files for selected longlat.
        #print('Looping over points')
        for point in points:
            
            print('Extracting parameter data for point ' + point['id'] + '.')
            # Loop over selected grib-files and create tabulars.
            for f in grib_files:
                f_name = f.split('.')[0]
                grib_f_dir = gribs_selected_dir + f_name + '.grib'
                tab_f_dir = tabs_selected_dir + f_name + '.tabular'
                print('Creating ' + f_name + '.tabular')
                !grib_get -l {point['lat']},{point['lon']},1  -W 50 -p dataDate {grib_f_dir} > {tab_f_dir}
        
            
            # Find all selected tabular_files.
            tab_files = []
            for r, d, f in os.walk(tabs_selected_dir):
                for file in f:
                    if '.tabular' in file:
                        tab_files.append(file)
            
            # Create new dataframe comb_df, insert datetimes.
            comb_df = pd.DataFrame()
            hours = []
            for ts in timestamps:
                if timestamps[ts]:
                    hours.append(ts)
            
            comb_df.insert(0, 'Timestamp', hours, allow_duplicates=False)
            # Loop over each tabular file:
            for f in tab_files:
                print('Reading ' + f)
                # Open as dataframes, add parameter value column to comb_df
                param = f.split('.tabular')[0].split('selected')[1][1:]
                #print(param)
                #df = pd.read_csv(directory + filename,  header=None, delim_whitespace=True, names=["Date", "Param"], index_col=0)
                df = pd.read_csv(tabs_selected_dir + f, header=None, names=["Date", "Param"], delim_whitespace=True)
    
                #print(df)
                #print(df['Param'])
                comb_df.insert(len(comb_df.columns), param, df['Param'], allow_duplicates=False)
            
            print(comb_df)
            
            # Save table as MESAN_DATE.csv in station folder.
            
            if os.path.isdir(csv_dir + point['id'] + '/'):
                pass
            else:
                print('Creating ' + csv_dir + point['id'] + '/' + 'directory.')
                os.mkdir(csv_dir + point['id'] + '/')
            
            print('Saving ' + 'MESAN_' + date_str + '.csv')
            comb_df.to_csv(csv_dir + point['id'] + '/' + 'MESAN_' + date_str + '.csv', index=False)
            
        
        current_date = current_date + datetime.timedelta(days=1)
        
        
# Write file with all recorded parameters as columns. Each row is a timestamp.
# Goal:
# TIMESTAMP            PARAM_1   PARAM_2   ...   PARAM_n
# 2020-11-13T00:00:00Z 0.0000    0.0000    ...   0.0000
# 2020-11-13T01:00:00Z 0.0000    0.0000    ...   0.0000
# 2020-11-13T02:00:00Z 0.0000    0.0000    ...   0.0000
#          ...           ...       ...     ...     ...
# 2020-11-13T23:00:00Z 0.0000    0.0000    ...   0.0000


In [10]:
# Example
points = [{'id': '149', 'lat': 65.59405, 'lon': 19.26423}]
start_date = datetime.date(2020, 9, 1)
end_date = datetime.date(2020, 9, 1)

GRIB_to_CSV(points, start_date, end_date)

Creating GRIBS/.
Creating GRIBS/Raw/.
Creating GRIBS/Selected/.
Creating TABS/
Creating METCOMP_CSV/
[]
Building GRIB-file.
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010000+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010100+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010200+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010300+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010400+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010500+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010600+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010700+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010800+000H00M
https://opendata-download-grid-archive.smhi.se/data/6/202009/MESAN_202009010900+000H